In [14]:
var target = {
  message1: "hello",
  message2: "everyone",
};

var handler3 = {
  get(target, prop, receiver) {
    console.log("get", prop, receiver);
    if (prop === "message2") {
      return "world";
    }
    return Reflect.get(...arguments);
  },
};

var proxy3 = new Proxy(target, handler3);

console.log(proxy3.message1); // hello
console.log(proxy3.message2); // world

get message1 { message1: 'hello', message2: 'everyone' }
hello
get message2 { message1: 'hello', message2: 'everyone' }
world


In [ ]:
proxy3[2, 3, ':'] // Can't work because the comma will resolve to the last expression.
proxy3[[2, 3, ':']] // The double brackets may confuse the user, AND there is no way to warn the user that he wrote proxy[2,3] instead of proxy[[2,3]]

proxy3([2, 3, ':']) // Could be...
proxy3(2, 3, ':') // Could be... but the user will be tempted to write proxy3(2, 3) = ..., and I need to know if target is source before calling `slice`

proxy3.get([2,3,':'])
proxy3.set(B, [2,[3],':'])
proxy3.set(B, [2,[3],':'], '+=')

proxy3.get(2, 3, ':')
proxy3.set(B, 2,3,':')
proxy3.set(B, '+=', 2,3,':') // But the type checker can't make the difference between a string literal '+=' and a string '1::'. PROBABLY



// CURRENT:
proxy3.slice([2,3,':'])
proxy3.op_assign['+='](B, 2,3,':')


// I don't like the word 'slice'
arr.index(2, 3, ':')
arr.assign(B, 2, 3, ':')
arr.assign['+='](B, 2, 3, ':')

// If I allow this or this:
arr.op['+'](B)
arr['+'](B)
// The user will expect this (OK):
arr.op['+='](B)
arr['+='](B)
// but also this (NOT ok for advanced indexing):
arr.index(1, '::2')['+='](B)
// Can I detect it? NO. This is legit (both if it's copy or view):
x = arr.index(1, '::2')
x['+='](B)

// Easiest, simplest SOLUTION:
arr.index(2, 3, ':')
arr.assign(B, 2, 3, ':')
arr.assign['+='](B, 2, 3, ':')
arr.assign['+='](B)
arr['+'](B)
// But not this:
arr['+='](B)

// OK, but the user will try this that will not work:
arr.index(2, [1, 3], ':').assign['+='](B)

// What about this?
arr.index(2, 3, ':', '=', B)
// Well, the semantics are lost and he may still write
arr.index(2, 3, ':').index('=', B)

// I honestly think that the best solution is to allow the user to assign, because he will try it no matter what notation I use, but to throw a warning:

arr.index(...simpleIndex)['+='](B) // Assigns.
arr.index(...advancedIndex)['+='](B) // Throws error.
x = arr.index(...advancedIndex)
x['+='](B) // Throws error
// If he wants to assign in place, he must do:
arr.assign['+='](B, advancedIndex)
arr.assign['+='](B, simpleIndex)
// Or:
arr['+='](B, advancedIndex)
arr['+='](B, simpleIndex)

// If he wants to index and assign to the copy, he must do:
x = arr.index(...advancedIndex).copy() // here, copy will not really copy again.
x['+='](B) // no error
x = arr.index(...simpleIndex).copy()
x['+='](B) // no error

// Everything is ok except that enabling x['+='] will disable type checking.
// I prefer this: (easier to read and no Proxy is needed)

x.op('+=', B, advancedIndex) // Here, I explicitly define the signature of every operator
x.op('~')
x.op('+', B, out)
// Maybe even
x.op('+', B, C, D) // (which disables out parameter) or
x.op('+', [B, C, D]) // NO. this will confuse if B,C,D are JS arrays. So maybe
x.op('+...', B, C, D)
// instead of
x.op('+', B).op('+', C).op('+', D)


// Will it make more sense to have this?

x = arr.view(...advancedIndex).op('+=', B)
// or simply
x = arr.index(...advancedIndex).op('+=', B)
// and
x = arr.index(...advancedIndex).copy()
// But I can't warn... Let's leave the proposal as it is...




